In [48]:
import pickle
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
import time
from selenium.common.exceptions import NoSuchElementException

# Function to fetch up to 100 reviews from the product URL and append to DataFrame
def fetch_reviews_amazon(url):
  

    # Set up Firefox WebDriver
    options = Options()
    options.headless = False  # Run Firefox in headless mode
    service = Service('geckodriver')  # Path to geckodriver executable
    driver = webdriver.Firefox(service=service, options=options)
    driver.get(url)
    time.sleep(2)
    see_all_reviews_link = driver.find_element(By.XPATH, "//a[@data-hook='see-all-reviews-link-foot']")
    see_all_reviews_link.click()
    print("fetching reviews")
    
    reviews = []

    # Fetch all review elements
    while len(reviews) < 100:
    # Fetch all review elements on the current page
        review_elements = driver.find_elements(By.CSS_SELECTOR, "[data-hook='review-body']")

    # Extract the text of each review element
        for review_element in review_elements:
            review_text = review_element.text.strip()
            reviews.append(review_text)

        # Break the loop if the maximum number of reviews has been reached
            if len(reviews) >= 100:
                break

    # Check if there is a next page button
        try:
            
    
            next_page_button = driver.find_element(By.LINK_TEXT, "Next page")
    # Click on the next page button if it exists
            next_page_button.click()
            time.sleep(3)
        except NoSuchElementException:
            break


    # Close the WebDriver
    driver.quit()

    df = pd.DataFrame({'review': reviews})

    return df

# Load the DataFrame with fetched reviews

# Example usage
product_url = 'https://www.amazon.in/Electronic-Spices-COMBO-Rosin-Soldering/dp/B07YX6SSPT/?_encoding=UTF8&pd_rd_w=dGLA6&content-id=amzn1.sym.04eccdc9-b3cd-4cc1-973f-6fc2a5bdb1a2&pf_rd_p=04eccdc9-b3cd-4cc1-973f-6fc2a5bdb1a2&pf_rd_r=AH9DBC9QQAVPJQ15ZC2W&pd_rd_wg=AfJYr&pd_rd_r=1ac5a20a-75c9-4c28-a022-067156b42fed&ref_=pd_gw_cr_wsim'
df = fetch_reviews_amazon(product_url)

# Save the updated DataFrame
df.to_csv('reviews.csv', index=False)


/tmp/ipykernel_50900/3494772590.py:16: DeprecationWarning: headless property is deprecated, instead use add_argument('-headless')
  options.headless = False  # Run Firefox in headless mode


fetching reviews


In [49]:
with open('classifier.pickle', 'rb') as file:
    classifier = pickle.load(file)

with open('vectorizer.pickle', 'rb') as file:
    vectorizer = pickle.load(file)

    
# Function to predict sentiment using the classifier and vectorizer
def predict_sentiment(text):
    # Transform the text using the vectorizer
    X = vectorizer.transform([text])

    # Predict the sentiment using the classifier
    sentiment = classifier.predict(X)

    return sentiment[0]

# Load the DataFrame with fetched reviews

# Apply sentiment prediction to each review in the DataFrame
df = pd.read_csv("reviews.csv")
df['review'] = df['review'].astype(str)
df['predicted_sentiment'] = df['review'].apply(predict_sentiment)

# Print the DataFrame with predicted sentiments
print(df.head())



                                              review predicted_sentiment
0  The lead and paste is of decent quality, the p...            Positive
1  Both wire and paste is of bad quality\nPaste i...            Positive
2  Bot the same product and very bad quality. Mis...            Negative
3  Average product. The combination of tin and le...            Positive
4                           Not upto my expectations            Positive


In [50]:
df['predicted_sentiment'].value_counts()

Positive    36
Negative    11
Neutral      7
Name: predicted_sentiment, dtype: int64